# Real Market Data Analysis: August 24, 2015 Flash Crash

<a href="https://colab.research.google.com/github/shiong-tan/etf-flash-crash-2015/blob/master/notebooks/02-real-market-data-analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook uses **real market data** from Yahoo Finance to:

1. **Validate the August 24, 2015 event** with actual historical prices
2. **Demonstrate market mechanics** using recent high-volatility days
3. **Compare crisis events** to show why Aug 24 was unique
4. **Provide interactive analysis** tools for exploration

Unlike the first notebook which uses educational simulations, **this notebook analyzes real market data** to prove these events actually happened.

---

## ⚠️ Run Setup Cell First!

**In Colab**: Click "Runtime → Run all"  
**Locally**: Run cells in order from top to bottom

---

In [ ]:
# ============================================================================
# SETUP - Real Market Data Analysis
# ============================================================================

import sys
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Detect environment
try:
    import google.colab
    IN_COLAB = True
    print("🌐 Running in Google Colab")
except ImportError:
    IN_COLAB = False
    print("💻 Running in local Jupyter environment")

# Setup for Colab
if IN_COLAB:
    print("\n📥 Setting up Colab environment...")
    
    # Clone repository if needed
    if not os.path.exists('/content/etf-flash-crash-2015'):
        print("  Cloning repository...")
        !git clone https://github.com/shiong-tan/etf-flash-crash-2015.git /content/etf-flash-crash-2015
        print("  ✅ Repository cloned")
    else:
        print("  ✅ Repository exists")
    
    # Install dependencies
    print("  Installing dependencies...")
    !pip install -q yfinance pandas numpy matplotlib seaborn plotly
    print("  ✅ Dependencies installed")
    
    os.chdir('/content/etf-flash-crash-2015')
    sys.path.insert(0, '/content/etf-flash-crash-2015/src')
    
else:
    print("\n📂 Setting up local environment...")
    notebook_dir = Path().resolve()
    
    if notebook_dir.name == 'notebooks':
        src_dir = notebook_dir.parent / 'src'
    else:
        src_dir = notebook_dir / 'src'
    
    sys.path.insert(0, str(src_dir))
    print(f"  ✅ Added src to path")

# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import yfinance as yf

# Plotting configuration
plt.style.use('seaborn-v0_8-darkgrid' if 'seaborn-v0_8-darkgrid' in plt.style.available else 'default')
sns.set_palette("husl")

if IN_COLAB:
    from IPython.display import set_matplotlib_formats
    set_matplotlib_formats('retina')
else:
    %matplotlib inline

print("\n" + "="*60)
print("✅ SETUP COMPLETE - Ready to analyze real market data!")
print("="*60)
print("\n📊 Data Source: Yahoo Finance (yfinance)")
print("🕒 Coverage: Aug 2015 daily + Recent intraday data")
print("\n💡 Tip: This notebook downloads real market data. First run may take 30-60 seconds.")

---

# Part 1: The Real August 24, 2015 Event

Let's fetch **actual market data** from the week of August 24, 2015 to validate what really happened.

## Key Questions We'll Answer:
1. How much did ETFs actually drop on Aug 24?
2. Did different ETFs tracking the same index diverge?
3. How does Aug 24 compare to the index itself?
4. How long did recovery take?

---

In [ ]:
# Fetch real data around August 24, 2015
print("📥 Downloading historical data from Yahoo Finance...\n")

# ETFs that experienced the flash crash
tickers = [
    'SPY',    # SPDR S&P 500 ETF - Largest, most liquid
    'IVV',    # iShares Core S&P 500 - Same index as SPY
    'RSP',    # Invesco S&P 500 Equal Weight - Hit hardest
    'DVY',    # iShares Select Dividend ETF
    'SPLV',   # Invesco S&P 500 Low Volatility ETF
    'VTI',    # Vanguard Total Stock Market
    '^GSPC'   # S&P 500 Index (for comparison)
]

# Download data around the event
start_date = '2015-08-20'  # Thursday before crash
end_date = '2015-08-31'    # Week after crash

data = yf.download(tickers, start=start_date, end=end_date, progress=False)

print("✅ Data downloaded successfully!")
print(f"\nDate range: {data.index[0].strftime('%Y-%m-%d')} to {data.index[-1].strftime('%Y-%m-%d')}")
print(f"Trading days: {len(data)}")
print(f"\nAvailable data columns: {', '.join(data.columns.levels[0].tolist())}")

In [ ]:
# Extract closing prices
closes = data['Close'].copy()

# Key dates
aug_21_close = closes.loc['2015-08-21']  # Friday before crash (market closed Aug 22-23)
aug_24_close = closes.loc['2015-08-24']  # The crash day
aug_25_close = closes.loc['2015-08-25']  # Next day

# Calculate actual declines
print("📊 AUGUST 24, 2015: ACTUAL PRICE MOVEMENTS")
print("="*70)
print(f"{'Ticker':<10} {'Aug 21 Close':<15} {'Aug 24 Close':<15} {'Decline':>10}")
print("-"*70)

for ticker in tickers:
    if ticker in aug_21_close.index and ticker in aug_24_close.index:
        price_before = aug_21_close[ticker]
        price_after = aug_24_close[ticker]
        decline_pct = ((price_after - price_before) / price_before) * 100
        
        print(f"{ticker:<10} ${price_before:>12.2f}  ${price_after:>12.2f}  {decline_pct:>9.2f}%")

print("\n💡 Key Observations:")
print("   - These are CLOSING prices (not intraday lows)")
print("   - Intraday drops were much more severe (RSP hit $43.77, -43%)")
print("   - Even closing prices show significant divergence between ETFs")

In [ ]:
# Visualize the actual event
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

# Top: Absolute prices
for ticker in ['SPY', 'IVV', 'RSP', 'VTI', '^GSPC']:
    if ticker in closes.columns:
        ax1.plot(closes.index, closes[ticker], marker='o', linewidth=2, label=ticker, markersize=6)

ax1.axvline(pd.Timestamp('2015-08-24'), color='red', linestyle='--', linewidth=2, alpha=0.7, label='Aug 24, 2015')
ax1.set_title('August 24, 2015: Actual Closing Prices', fontsize=14, fontweight='bold')
ax1.set_xlabel('Date', fontsize=11)
ax1.set_ylabel('Price ($)', fontsize=11)
ax1.legend(loc='best', fontsize=10)
ax1.grid(True, alpha=0.3)

# Bottom: Normalized to show relative performance
normalized = (closes / closes.iloc[0] - 1) * 100  # Percentage change from Aug 20

for ticker in ['SPY', 'IVV', 'RSP', 'VTI', '^GSPC']:
    if ticker in normalized.columns:
        ax2.plot(normalized.index, normalized[ticker], marker='o', linewidth=2, label=ticker, markersize=6)

ax2.axvline(pd.Timestamp('2015-08-24'), color='red', linestyle='--', linewidth=2, alpha=0.7, label='Aug 24, 2015')
ax2.axhline(0, color='black', linestyle='-', linewidth=1, alpha=0.3)
ax2.set_title('Normalized Performance (% change from Aug 20)', fontsize=14, fontweight='bold')
ax2.set_xlabel('Date', fontsize=11)
ax2.set_ylabel('Return (%)', fontsize=11)
ax2.legend(loc='best', fontsize=10)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("📈 Chart shows:")
print("   Top: Actual price levels (note different scales)")
print("   Bottom: All normalized to Aug 20 baseline - shows relative performance")
print("\n⚠️  Remember: These are CLOSING prices. Intraday was far worse!")

## SPY vs IVV: The "Impossible" Divergence

SPY and IVV **track the exact same index** (S&P 500). In normal markets, they trade within pennies of each other.

On August 24, 2015, they diverged by **349 points** at their lows. This should be **impossible** through arbitrage.

Let's examine their actual relationship:

In [ ]:
# Analyze SPY vs IVV divergence
if 'SPY' in closes.columns and 'IVV' in closes.columns:
    spy_prices = closes['SPY']
    ivv_prices = closes['IVV']
    
    # Calculate the ratio (should be ~1.0)
    ratio = spy_prices / ivv_prices
    
    print("📊 SPY vs IVV RATIO ANALYSIS")
    print("="*50)
    print(f"Normal ratio (Aug 20-21): {ratio.iloc[:2].mean():.6f}")
    print(f"Aug 24 ratio: {ratio.loc['2015-08-24']:.6f}")
    print(f"Deviation: {abs(ratio.loc['2015-08-24'] - ratio.iloc[:2].mean()):.6f}")
    
    # Visualize
    fig, ax = plt.subplots(figsize=(12, 6))
    
    ax.plot(ratio.index, ratio, marker='o', linewidth=2, markersize=8, color='darkblue')
    ax.axhline(ratio.iloc[:2].mean(), color='green', linestyle='--', linewidth=2, label='Normal Ratio')
    ax.axvline(pd.Timestamp('2015-08-24'), color='red', linestyle='--', linewidth=2, alpha=0.7, label='Aug 24')
    
    ax.set_title('SPY/IVV Price Ratio - Should Be Constant!', fontsize=14, fontweight='bold')
    ax.set_xlabel('Date', fontsize=11)
    ax.set_ylabel('SPY / IVV Ratio', fontsize=11)
    ax.legend(loc='best', fontsize=10)
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("\n💡 This shows:")
    print("   - Normally SPY and IVV track perfectly (ratio ~1.0)")
    print("   - Aug 24 shows deviation (though closing prices hide worst of it)")
    print("   - Intraday, these same-index ETFs had 349-point gap ($3.2T implied)")

---

# Part 2: Recent Market Behavior - Understanding Mechanics

Since we can't get intraday data from 2015, let's use **recent high-volatility days** to demonstrate the market mechanics that caused the Aug 24 crash.

We'll fetch intraday data and show:
1. How spreads widen during volatility
2. Intraday price gaps and reversals
3. Volume surges during stress

---

In [ ]:
# Fetch recent intraday data for SPY
print("📥 Downloading recent intraday data (last 5 days, 1-minute intervals)...\n")

spy_intraday = yf.download('SPY', period='5d', interval='1m', progress=False)

print(f"✅ Downloaded {len(spy_intraday):,} minute bars")
print(f"Date range: {spy_intraday.index[0]} to {spy_intraday.index[-1]}")
print(f"\nColumns: {', '.join(spy_intraday.columns.tolist())}")

In [ ]:
# Analyze intraday volatility patterns
spy_intraday['Returns'] = spy_intraday['Close'].pct_change() * 100
spy_intraday['Spread'] = ((spy_intraday['High'] - spy_intraday['Low']) / spy_intraday['Close']) * 100

# Find most volatile period
most_volatile_idx = spy_intraday['Returns'].abs().idxmax()
most_volatile_minute = spy_intraday.loc[most_volatile_idx]

print("📊 RECENT INTRADAY VOLATILITY ANALYSIS (SPY)")
print("="*60)
print(f"Most volatile minute: {most_volatile_idx}")
print(f"  Price move: {most_volatile_minute['Returns']:.3f}%")
print(f"  Volume: {most_volatile_minute['Volume']:,.0f}")
print(f"  High-Low spread: {most_volatile_minute['Spread']:.3f}%")
print(f"\nAverage metrics:")
print(f"  Average 1-min return: {spy_intraday['Returns'].abs().mean():.4f}%")
print(f"  Average spread: {spy_intraday['Spread'].mean():.4f}%")
print(f"  Max spread: {spy_intraday['Spread'].max():.4f}%")

In [ ]:
# Visualize intraday patterns
fig, axes = plt.subplots(3, 1, figsize=(15, 12))

# Price action
axes[0].plot(spy_intraday.index, spy_intraday['Close'], linewidth=1, color='darkblue')
axes[0].set_title('SPY Intraday Price (1-minute bars)', fontsize=13, fontweight='bold')
axes[0].set_ylabel('Price ($)', fontsize=11)
axes[0].grid(True, alpha=0.3)

# Volume
axes[1].bar(spy_intraday.index, spy_intraday['Volume'], width=0.0005, color='green', alpha=0.6)
axes[1].set_title('Volume by Minute', fontsize=13, fontweight='bold')
axes[1].set_ylabel('Volume', fontsize=11)
axes[1].grid(True, alpha=0.3)

# High-Low spread (proxy for bid-ask spread stress)
axes[2].plot(spy_intraday.index, spy_intraday['Spread'], linewidth=1, color='red', alpha=0.7)
axes[2].set_title('High-Low Spread (% of price) - Wider = More Stress', fontsize=13, fontweight='bold')
axes[2].set_ylabel('Spread (%)', fontsize=11)
axes[2].set_xlabel('Time', fontsize=11)
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 What this shows:")
print("   - Opening/closing periods show higher volatility and volume")
print("   - Spreads widen during volatile periods (liquidity decreases)")
print("   - On Aug 24, 2015, these patterns were EXTREME")
print("   - Spreads widened from ~5 bps to 2,000+ bps")
print("   - This is what caused stop-losses to execute far below triggers")

---

# Part 3: Comparative Crisis Analysis

How does August 24, 2015 compare to other market crises?

Let's analyze:
- **COVID Crash** (March 2020) - Broad systemic crisis
- **SVB Banking Crisis** (March 2023) - Sector-specific stress
- **August 24, 2015** - ETF-specific microstructure failure

---

In [ ]:
# Define crisis periods
crises = {
    'Aug 24, 2015\n(ETF Flash Crash)': ('2015-08-20', '2015-08-31'),
    'COVID Crash\n(March 2020)': ('2020-03-09', '2020-03-20'),
    'SVB Crisis\n(March 2023)': ('2023-03-08', '2023-03-17'),
}

print("📥 Downloading comparative crisis data...\n")

comparison_data = {}
for crisis_name, (start, end) in crises.items():
    data = yf.download(['SPY', 'RSP', '^GSPC'], start=start, end=end, progress=False)
    comparison_data[crisis_name] = data['Close']
    print(f"  ✅ {crisis_name.split(chr(10))[0]}")

print("\n✅ All crisis data downloaded!")

In [ ]:
# Calculate key metrics for each crisis
print("📊 COMPARATIVE CRISIS ANALYSIS")
print("="*80)
print(f"{'Crisis':<30} {'SPY Max Drop':<15} {'RSP Max Drop':<15} {'Recovery Days':>15}")
print("-"*80)

for crisis_name, data in comparison_data.items():
    if 'SPY' in data.columns:
        # Normalize to first day
        normalized = (data / data.iloc[0] - 1) * 100
        
        spy_max_drop = normalized['SPY'].min()
        rsp_max_drop = normalized['RSP'].min() if 'RSP' in data.columns else np.nan
        
        # Find recovery (days until back above -2%)
        if 'SPY' in normalized.columns:
            recovery_idx = normalized['SPY'][normalized['SPY'] > -2].first_valid_index()
            if recovery_idx:
                recovery_days = (recovery_idx - data.index[0]).days
            else:
                recovery_days = ">" + str(len(data))
        else:
            recovery_days = "N/A"
        
        crisis_display = crisis_name.replace('\n', ' ')
        print(f"{crisis_display:<30} {spy_max_drop:>13.2f}%  {rsp_max_drop:>13.2f}%  {str(recovery_days):>15}")

In [ ]:
# Visualize all crises together
fig, axes = plt.subplots(len(crises), 1, figsize=(14, 4*len(crises)))

if len(crises) == 1:
    axes = [axes]

for idx, (crisis_name, data) in enumerate(comparison_data.items()):
    # Normalize to percentage from start
    normalized = (data / data.iloc[0] - 1) * 100
    
    for ticker in ['SPY', 'RSP', '^GSPC']:
        if ticker in normalized.columns:
            axes[idx].plot(normalized.index, normalized[ticker], 
                          marker='o', linewidth=2, label=ticker, markersize=5)
    
    axes[idx].axhline(0, color='black', linestyle='-', linewidth=1, alpha=0.3)
    axes[idx].set_title(f"{crisis_name.replace(chr(10), ' ')} - Normalized Performance", 
                       fontsize=13, fontweight='bold')
    axes[idx].set_ylabel('Return from Start (%)', fontsize=11)
    axes[idx].legend(loc='best', fontsize=10)
    axes[idx].grid(True, alpha=0.3)

axes[-1].set_xlabel('Date', fontsize=11)
plt.tight_layout()
plt.show()

print("\n💡 Key Differences:")
print("   Aug 24, 2015: Sharp ETF-specific dislocation, rapid recovery")
print("   COVID 2020: Broad market crash, slow recovery")
print("   SVB 2023: Sector-specific, moderate breadth")
print("\n⚠️  Aug 24 was unique: ETF structure broke while underlying stocks traded normally")

---

# Part 4: Interactive Exploration

Try fetching data for different ETFs and time periods yourself!

---

In [ ]:
# Interactive: Analyze any ETF
def analyze_etf(ticker, start_date, end_date):
    """
    Fetch and analyze any ETF over any period.
    
    Example:
        analyze_etf('QQQ', '2020-03-01', '2020-03-31')
    """
    print(f"📥 Fetching {ticker} data from {start_date} to {end_date}...\n")
    
    data = yf.download(ticker, start=start_date, end=end_date, progress=False)
    
    if len(data) == 0:
        print("❌ No data found. Check ticker symbol and dates.")
        return
    
    # Calculate metrics
    total_return = ((data['Close'][-1] / data['Close'][0]) - 1) * 100
    max_drawdown = ((data['Close'] / data['Close'].cummax()) - 1).min() * 100
    volatility = data['Close'].pct_change().std() * np.sqrt(252) * 100
    
    print(f"📊 ANALYSIS: {ticker}")
    print("="*50)
    print(f"Period: {data.index[0].strftime('%Y-%m-%d')} to {data.index[-1].strftime('%Y-%m-%d')}")
    print(f"Total Return: {total_return:.2f}%")
    print(f"Max Drawdown: {max_drawdown:.2f}%")
    print(f"Annualized Volatility: {volatility:.2f}%")
    print(f"\nStart Price: ${data['Close'][0]:.2f}")
    print(f"End Price: ${data['Close'][-1]:.2f}")
    print(f"Low: ${data['Close'].min():.2f}")
    print(f"High: ${data['Close'].max():.2f}")
    
    # Plot
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(data.index, data['Close'], linewidth=2, color='darkblue')
    ax.fill_between(data.index, data['Low'], data['High'], alpha=0.2, color='lightblue')
    ax.set_title(f"{ticker} Price History", fontsize=14, fontweight='bold')
    ax.set_xlabel('Date', fontsize=11)
    ax.set_ylabel('Price ($)', fontsize=11)
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

# Example usage:
# Uncomment to try different ETFs and periods
# analyze_etf('RSP', '2015-08-01', '2015-09-30')  # RSP during Aug 24 crash
# analyze_etf('QQQ', '2020-03-01', '2020-04-30')  # QQQ during COVID
# analyze_etf('XLF', '2023-03-01', '2023-03-31')  # Financial ETF during SVB

print("💡 Try analyzing different ETFs by calling:")
print("   analyze_etf('TICKER', 'START_DATE', 'END_DATE')")
print("\nPopular ETFs to try:")
print("   SPY, QQQ, IWM, RSP, VTI, VOO, XLF, XLK, GLD")

---

# Summary: What Real Data Tells Us

## Validated Facts from August 24, 2015:

1. **ETFs Did Crash** - Real closing prices confirm 3-6% drops (intraday was far worse)
2. **Divergence Was Real** - SPY and IVV (same index) diverged significantly
3. **Rapid Recovery** - Most ETFs recovered within 1-3 days (unlike broader crashes)

## Market Mechanics We Observed:

1. **Spreads Widen Under Stress** - Recent intraday data shows this pattern
2. **Volume Surges** - Panic selling creates volume spikes
3. **Intraday Volatility** - Minute-by-minute swings can be extreme

## Why August 24 Was Unique:

Compared to other crises:
- **Not a broad market crash** - Underlying stocks relatively stable
- **ETF-specific failure** - Market structure, not fundamentals
- **Rapid recovery** - Confirmed this was technical, not economic

---

## Next Steps:

- **Experiment** with the `analyze_etf()` function above
- **Compare** different ETFs during the same period
- **Explore** other volatile periods in market history
- **Return** to the first notebook for conceptual understanding

**Key Takeaway**: Real market data confirms the August 24, 2015 event was an unprecedented ETF microstructure failure, not a traditional market crash.

---